# [Module 3.4.1] Train Docker Image 생성 및 ECR 퍼블리시

이 노트북은 Docker 이미지를 생성하고, Amazon ECR(Elastic Container Registry)에 퍼블리스를 함

---
노트북의 소요 시간은 약 2분 걸립니다.

---

Docker Info:
- Get Started: Build Your Custom Training Container with Amazon SageMaker
    - https://docs.aws.amazon.com/sagemaker/latest/dg/build-container-to-train-script-get-started.html
- Built-in Container Image
    - https://github.com/aws/deep-learning-containers/blob/master/available_images.md

## Train Script를 다커 폴더에 복사

In [1]:
bert_train_file = "tf_script_bert_tweet.py"
! cp {bert_train_file} "train_container/"

## train_container 폴더의 Dockerfile 

In [1]:
! pygmentize train_container/Dockerfile


# 기존의  Pre-built-in TF2.1-gpu image를 가져옴
FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.1.1-gpu-py36-cu101-ubuntu18.04

# transformers 설치
RUN pip install transformers==2.8.0


ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

ENV PATH="/opt/ml/code:${PATH}"

# Copy training code
COPY tf_script_bert_tweet.py /opt/ml/code/
 
WORKDIR /opt/ml/code

ENV SAGEMAKER_PROGRAM tf_script_bert_tweet.py


# Train Docker 이미지를 ECR (Elastic Container Registry) 에 Push

In [3]:
import os
os.environ['train_container_name']= "bert2tweet"

In [4]:
%%sh
cd train_container

# The name of our algorithm
algorithm_name=$train_container_name


account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

echo $region

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the Tensorflow-gpu:1.5 image
$(aws ecr get-login --registry-ids 763104351884 --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

us-west-2
Login Succeeded
Login Succeeded
Sending build context to Docker daemon  24.58kB
Step 1/8 : FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.1.1-gpu-py36-cu101-ubuntu18.04
 ---> edb9e75607cd
Step 2/8 : RUN pip install transformers==2.8.0
 ---> Using cache
 ---> 75d023be87dd
Step 3/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 23f08cc885be
Step 4/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 7d9c14c7c001
Step 5/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 19f53ffeb739
Step 6/8 : COPY tf_script_bert_tweet.py /opt/ml/code/
 ---> Using cache
 ---> 9475e6a27ae5
Step 7/8 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 56d3a59ef082
Step 8/8 : ENV SAGEMAKER_PROGRAM tf_script_bert_tweet.py
 ---> Using cache
 ---> 73d0d642b665
[Warning] One or more build-args [REGION] were not consumed
Successfully built 73d0d642b665
Successfully tagged bert2tweet:latest
The push refers to repository [057716757052.dkr.ecr.us-west-2.amazonaws.co

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

